<IMG src="images/title.jpg">

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoNormal><span style='font-size:48.0pt'>Notebook Technologies<o:p></o:p></span></p>

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoListParagraphCxSpFirst style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Development<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Sharing<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Answers<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpLast style='margin-left:40.5pt;mso-add-space:auto;
text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Presentations?<o:p></o:p></span></p>

<IMG src="images/internal_monologues.png">

<IMG src="images/intro.jpg">

<IMG src="images/notebooks.jpg">


<IMG src="images/useCases.jpg">

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoNormal><span style='font-size:48.0pt'>Mapping events - the long way<o:p></o:p></span></p>

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoListParagraphCxSpFirst style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Carve GeoLite2 City into a dataframe<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Trim the IDS logs based on directionality<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Iterate events - Lookup long/lat<o:p></o:p></span></p>

In [1]:
import ipaddress

def string_to_ip(ipstring):
    return ipaddress.ip_network(ipstring)

In [5]:
#slow

import pandas

from zipfile import ZipFile
with ZipFile('./Downloads/GeoLite2-City-CSV_20181113.zip') as zip:
    df_maxmind = pandas.read_csv(zip.open('GeoLite2-City-CSV_20181113/GeoLite2-City-Blocks-IPv4.csv'))
    
df_maxmind = df_maxmind.reset_index()
df_maxmind['network'] = df_maxmind['network'].apply(string_to_ip)


In [6]:
df_maxmind.head(5)

,index,network,geoname_id,registered_country_geoname_id,represented_country_geoname_id,is_anonymous_proxy,is_satellite_provider,postal_code,latitude,longitude,accuracy_radius
0,0,1.0.0.0/24,2077456.0,2077456.0,NaN,0,0,NaN,-33.4940,143.2104,1000.0
1,1,1.0.1.0/24,1811017.0,1814991.0,NaN,0,0,NaN,24.4798,118.0819,50.0
2,2,1.0.2.0/23,1811017.0,1814991.0,NaN,0,0,NaN,24.4798,118.0819,50.0
3,3,1.0.4.0/22,2077456.0,2077456.0,NaN,0,0,NaN,-33.4940,143.2104,1000.0
4,4,1.0.8.0/21,1809935.0,1814991.0,NaN,0,0,NaN,23.1167,113.2500,50.0


In [10]:
import os
import csv

events = []
directory = 'ids'
for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith('.csv') and file.startswith('cisco'):
            with open(directory+'/'+file) as csvfile:
                idscsv = csv.reader(csvfile)
                next(idscsv)
                for row in idscsv:
                    events.append((row[3],row[6]))

In [12]:
from collections import Counter
bad_guys = []

for idstuple in events:
    if idstuple[0].startswith('40.'):
        if idstuple[1].startswith('40.'):
            continue
        else:
            bad_guys.append(idstuple[1])
    else:
        bad_guys.append(idstuple[0])

worst_guys = {}
for key,value in dict(Counter(bad_guys)).items():
    worst_guys[key] = {}
    worst_guys[key]['count'] = value

<font size="10"><p style="text-align: center;">OMG NO!</p></font>

In [ ]:
for bad_guy in worst_guys:
    print(bad_guy)
    df_temp = df_maxmind[df_maxmind.network.apply(lambda x: ipaddress.ip_address(bad_guy) in x)][["longitude","latitude"]]
    df_temp = df_temp.reset_index()
    worst_guys[bad_guy]["longitude"] = df_temp.at[0,"longitude"]
    worst_guys[bad_guy]["latitude"] = df_temp.at[0,"longitude"]

In [14]:
print('\U0001F4A9')

💩


In [ ]:
from IPython.display import Audio
Audio(filename='images/My Song.wav', autoplay=True)

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoNormal><span style='font-size:48.0pt'>Mapping events - the short way<o:p></o:p></span></p>

<p class=MsoNormal><span style='font-size:24.0pt'><o:p>&nbsp;</o:p></span></p>

<p class=MsoListParagraphCxSpFirst style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Use GeoLite2 library and database<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Use the trimmed IDS logs<o:p></o:p></span></p>

<p class=MsoListParagraphCxSpMiddle style='margin-left:40.5pt;mso-add-space:
auto;text-indent:-.5in;mso-list:l1 level1 lfo3'><span
style='font-size:36.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
mso-bidi-font-family:Symbol'><span style='mso-list:Ignore'>·<span
style='font:7.0pt "Times New Roman"'>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
</span></span></span><span style='font-size:36.0pt'>Push long and lat into dictionary<o:p></o:p></span></p>

In [15]:
#https://dev.maxmind.com/geoip/geoip2/geolite2/
#This product includes GeoLite2 data created by MaxMind, available from
#<a href="http://www.maxmind.com">http://www.maxmind.com</a>
import geoip2.database
reader = geoip2.database.Reader('Downloads/GeoLite2-City.mmdb')

In [16]:
for bad_guy in worst_guys:
    #print(bad_guy)
    response = reader.city(bad_guy)
    worst_guys[bad_guy]["longitude"] = round(response.location.longitude,3)
    worst_guys[bad_guy]["latitude"] = round(response.location.latitude,3)

In [17]:
location = []

for bad_guy,values in worst_guys.items():
    location.append((values['latitude'],values['longitude']))
    
location = set(location)

In [18]:
import pickle

with open('/Users/c266770/Desktop/pickles/api.pkl', 'rb') as f:
    api = pickle.load(f)
    
def save_obj(obj, name):
    with open(pickle_dir + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(pickle_dir + str(name) + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
import gmaps
import gmaps.datasets

gmaps.configure(api['googleMaps'])

In [ ]:
fig = gmaps.figure(center=(0,0),zoom_level=1.75, map_type='HYBRID')
heatmap = gmaps.heatmap_layer(locations=location)
fig.add_layer(heatmap)

In [ ]:
fig

<IMG src="images/war_game.gif" width=800>

Tangent - Vela incident

In [ ]:
import os
import csv
import pandas
import matplotlib.pyplot as plt

In [ ]:
directory = '../reports/av'
events = []

for root,dirs,files in os.walk(directory):
    for file in files:
        if file.endswith('.csv') and file.startswith('av_daily'):
            date = file.split('_')[2].split('.')[0].split('-')[0:3]
            date = "-".join(date)
            with open(directory+'/'+file) as csvfile:
                devicescsv = csv.reader(csvfile)
                for row in devicescsv:
                    if len(row[0]) ==32:
                        row.append(date)
                        events.append(row)

In [ ]:
labels = ['col1','col2','col3','col4','col5','col6','col7','col8','col9','col10']
df_avReports = pandas.DataFrame.from_records(events, columns=labels)

df_mcafeeAv = df_avReports[df_avReports.col2 == 'mcafee_av'][['col1','col4','col5','col8']]
df_mcafeeAv = df_mcafeeAv.rename(index=str, columns={'col1':'md5','col4':'src_host','col5':'av_sig','col8':'date'})
df_mcafeeAv['date'] = pandas.to_datetime(df_mcafeeAv['date'])
#df_mcafeeAv.head(5)

df_mcafeeHip = df_avReports[df_avReports.col2 == 'mcafee_hip'][['col1','col5','col9','col10']]
df_mcafeeHip = df_mcafeeHip.rename(index=str, columns={'col1':'md5','col5':'src_host','col9':'av_sig','col10':'date'})
df_mcafeeHip['date'] = pandas.to_datetime(df_mcafeeHip['date'])
#df_mcafeeHip.head(5)

df_cylance = df_avReports[df_avReports.col2 == 'cylance_av'][['col1','col5','col3','col7']]
df_cylance.col1 = df_cylance.col1.map(lambda x: x.lower())
df_cylance = df_cylance.rename(index=str, columns={'col1':'md5','col5':'src_host','col3':'av_sig','col7':'date'})
df_cylance['date'] = pandas.to_datetime(df_cylance['date'])
#df_cylance.head(5)

dfs = [df_mcafeeAv, df_mcafeeHip, df_cylance]
df_avcombo = pandas.concat(dfs)
df_avcombo.head(5)

In [ ]:
y = df_avcombo.groupby('date').count()['md5'].map(lambda x: float(x))
df_y = pandas.DataFrame(y)
df_y = df_y.asfreq('D')
start = len(df_y) -10
end = len(df_y) + 15

In [ ]:
import statsmodels.api as sm

mod = sm.tsa.statespace.SARIMAX(df_y,
                                order=(0, 1, 1),
                                seasonal_order=(1, 1, 1, 7),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()

In [ ]:
pred_results = results.predict(start=start, end=end, dynamic=True)

ax = df_y.plot(label='observed', figsize=(20, 15))
pred_results.plot(ax=ax, label='forecast')
plt.legend()

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
import math

output_notebook()

## You doing ok?

## a: I'm doing bokeh

In [ ]:
plot = figure(x_axis_type='datetime', plot_width = 800)
source = ColumnDataSource(df_y)

df_pred = pred_results.to_frame().reset_index().rename(columns={'index':'date',0:"md5"}).set_index('date')
source2 = ColumnDataSource(df_pred)

plot.line('date', 'md5', source=source)
plot.line('date', 'md5', source = source2, line_color = 'red')
plot.xaxis.major_label_orientation = math.pi/2

show(plot)

# API

In [20]:
%run /Users/c266770/Desktop/notebooks/proxy-load.ipynb

In [ ]:
import requests

def vt_call(hashes):
    from time import sleep 
    
    if type(hashes) == list:
        if len(hashes) < 5:
            resource = ','.join(hashes)
                
        elif len(hashes) >= 5:
            resource = [hashes[i:i + 4] for i in range(0, len(hashes), 4)]
        else:
            return 'empty list?'
    elif type(hashes) == str and len(hashes.split(',')) == 1:
        resource = hashes
    elif type(hashes) == str and len(hashes.split(',')) < 4:
        resource = hashes
    elif type(hashes) == str and len(hashes.split(',')) > 4:
        hashes = hashes.split(',')
        resource = [hashes[i:i + 4] for i in range(0, len(hashes), 4)]
    else:
        return ''
    
    headers = {"Accept-Encoding": "gzip, deflate",
               "User-Agent" : "python"
              }
    
    if type(resource) != list:
        params = {'apikey': api['vt'], 'resource': resource}
        response = requests.get('https://www.virustotal.com/vtapi/v2/file/report',
                                params = params, 
                                headers = headers,
                                proxies = proxies)
        return response.json()
    else:
        response = []
        for hash_list in resource:
            params = {'apikey': api['vt'], 'resource': ','.join(hash_list)}
            vt_request = requests.get('https://www.virustotal.com/vtapi/v2/file/report',
                                      params = params,
                                      headers = headers,
                                      proxies = proxies)
            new_stuff = vt_request.json()
            if type(new_stuff) == dict:
                temp_list = []
                temp_list.append(new_stuff)
                response = response + temp_list
            else:
                response = response + new_stuff
            
        return response

In [ ]:
test = ['866fcd320412a9fdd78aefc31372c74e',
        '29c1a72fa565f339065d1f6c8a58de11',
        '38acd35c1b04167823c83bbd8a12352c',
        '28b78a1ab9f541a0a85cb853eb556fe8',
        'ac8c1fa592b1551fd57072c371b7c748']

In [ ]:
for hash in vt_call(test):
    print(hash['md5'],hash['positives'])

In [ ]:
!docker run -p 8080:8080 --rm \
    -v $PWD/logs:/logs -v $PWD/notebook:/notebook \
    -e ZEPPELIN_LOG_DIR='/logs' \
    -e ZEPPELIN_NOTEBOOK_DIR='/notebook' \
    --name zeppelin apache/zeppelin:0.8.0